In [1]:
from pyspark.sql import *
from pyspark.sql.types import *
from time import time
from functools import reduce
from datetime import date
from GetPath import *
import shutil

In [2]:
# Config
input_path='D:/csv_enhealth/Raw/patient_information/'
output_path='D:/Output/chenbingying/A1/'

In [3]:
# 生日处理
def Birth(x,y): # x为生日，y为ETL_Date
    try:
        if((y-x).days<0 or (x-date(1900,1,1)).days<0):
            return None
        else:
            return x
    except:
        return None

In [4]:
# 性别处理
def Gender(x,y): # x为Gender,y为GenderName
    if(x is not None):
        if(x==2 or x=='2' or x=='F' or x=='2.0' or '女' in x):
            return '2'
        elif(x==1 or x=='1' or x=='M' or x=='1.0'  or '男' in x):
            return '1'
        else:
            if(y is not None):
                if(y==2 or y=='2' or y=='F' or y=='2.0' or '女' in y):
                    return '2'
                elif(y==1 or y=='1' or y=='M' or y=='1.0' or '男' in y):
                    return '1'
            else:
                return None        
    else:
        if(y is not None):
            if(y==2 or y=='2' or y=='F' or y=='2.0' or '女' in y):
                return '2'
            elif(y==1 or y=='1' or y=='M' or y=='1.0' or '男' in y):
                return '1'
            else:
                return None
        else:
            return None

In [5]:
# 性别名处理
def GenderName(x,y): # x为GenderName,y为Gender
    if(x is not None):
        if(x==2 or x=='2' or x=='F' or x=='2.0' or '女' in x):
            return '女'
        elif(x==1 or x=='1' or x=='M' or x=='1.0'  or '男' in x):
            return '男'
        else:
            if(y is not None):
                if(y==2 or y=='2' or y=='F' or y=='2.0' or '女' in y):
                    return '女'
                elif(y==1 or y=='1' or y=='M' or y=='1.0' or '男' in y):
                    return '男'
            else:
                return None        
    else:
        if(y is not None):
            if(y==2 or y=='2' or y=='F' or y=='2.0' or '女' in y):
                return '女'
            elif(y==1 or y=='1' or y=='M' or y=='1.0' or '男' in y):
                return '男'
            else:
                return None
        else:
            return None

In [6]:
# 地址处理
def Address(x):
    if(x is None):
        return None
    if (len(x) <1) or (x in ['不详', '未录入', '无', '不', '本院', '不填', 'n', 'null', 'x', 'X', 'NULL', 'Null', 'N', '/', '.']) or ('本级编码' in x) or ('医院无此信息' in x) :
        return None
    else:
        for i in [['芳村区','荔湾区'],['花县','花都区'],['东山区','越秀区'],['东山口','越秀区'],['萝岗区','黄埔区']]:
            x=x.replace(i[0],i[1])  
        if(x in ['广州','本市','广州市']):
            return '广东省广州市'
        if(x[:2] in ['越秀','海珠','荔湾','天河','白云','黄埔','南沙','番禺','花都','增城','从化','芳村']):
            return '广东省广州市' + x
        else:
            return x

In [7]:
# 地址填补
def Fill_Address(x,y):
    if(x is not None):
        return x
    else:
        return y

In [8]:
# 去空格
def Cut_Space(x):
    if(x is None):
        return None
    else:
        x = str(x).replace(' ','')
        if(x == ''):
            return None
        else:
            return str(x)

In [9]:
# 婚姻状况名称填补
def Fill_MaritalStatusName(x,y):
    if(y is not None):
        return y
    else:
        if(x is None):
            return None
        x=str(x)
        if('.' in x):
            x=x.split('.')[0]
        x1=['90','10','20','21','22','23','30','40']
        x2=['未说明的婚姻状况','未婚','已婚','初婚','再婚','复婚','丧偶','离婚']
        for i in range(len(x1)):
            if(x==x1[i]):
                return x2[i]
        return None

In [10]:
# 联系人关系名称填补
def Fill_RelationshipName(x,y):
    if(y is not None):
        return y
    else:
        if(x is None):
            return None
        x=str(x)
        if('.' in x):
            x=x.split('.')[0]
        x1=['0','1','10','11','12','2','20','21','22','23','3','30','31','32','33','4','40','41','42','43','44','45','46','47','48','49','5','50','51','52','53','54','55','56','57','58','59','6','60','61','62','63','64','65','66','67','68','69','7','70','71','72','73','74','75','76','77','78','79','8','80','81','82','83','84','85','86','87','88','89','90','91','92','93','94','95','96','97','98','99']
        x2=['本人','配偶','其他情况配偶','夫','妻','子','儿子','养子','女婿','其他儿子','女','女儿','养女','儿媳','其他女儿','孙子、孙女或外孙子、外孙女','孙子','孙子','孙女','外孙子','外孙女','孙媳妇','孙女婿','曾孙子','曾孙女','其他孙子','父母','父母','父亲','母亲','公公','婆婆','岳父','岳母','继父','继母','其他父母关系','祖父母或外祖父母','祖父母','祖父','祖母','外祖父','外祖母','配偶的祖父母','曾祖父','曾祖母','配偶的曾祖父母','其他祖父母','兄、弟、姐、妹','兄弟姐妹','兄','嫂','弟','弟媳','姐姐','姐夫','妹妹','妹夫','其他兄弟姐妹','其他','其他','伯父','伯母','叔父','婶母','舅父','舅母','姨父','姨母','姑父','姑母','堂兄弟','表兄弟','侄子','侄女','外甥','外甥女','其他亲属','保姆','非亲属']
        for i in range(len(x1)):
            if(x==x1[i]):
                return x2[i]
        return None

In [11]:
# 职业分类名称填补
def Fill_OccupationalCategoryName(x,y):
    if(y is not None):
        return y
    else:
        if(x is None):
            return None
        x=str(x)
        if('.' in x):
            x=x.split('.')[0]
        x1=['Y','Y-0','Y-00','0','0-1','0-10','0-2','0-21','0-22','0-23','0-24','0-25','0-29','0-3','0-31','0-32','0-33','0-39','0-4','0-41','0-42','0-43','0-49','0-5','0-50','1','2','1-1','1-2','1-11','1-12','1-13','1-14','1-15','1-16','1-17','1-18','1-19','1-21','1-22','1-23','1-24','1-25','1-26','1-27','1-28','1-29','1-3','1-4','1-5','1-6','1-31','1-32','1-33','1-34','1-35','1-36','1-37','1-38','1-39','1-41','1-42','1-43','1-44','1-45','1-46','1-47','1-48','1-49','1-51','1-52','1-53','1-54','1-55','1-56','1-57','1-58','1-59','1-61','1-62','1-63','1-64','1-65','1-66','1-67','1-69','1-7','1-71','1-72','1-73','1-74','1-75','1-76','1-79','1-8','1-81','1-82','1-89','1-9','1-91','1-92','1-93','1-94','1-95','1-96','1-97','1-98','1-99','2-1','2-11','2-12','2-13','2-14','2-15','2-19','2-2','2-21','2-22','2-23','2-29','2-3','2-31','2-32','2-33','2-34','2-35','2-36','2-39','2-4','2-41','2-42','2-43','2-44','2-45','2-46','2-49','2-5','2-51','2-52','2-53','2-54','2-55','2-56','2-57','2-59','2-6','2-60','2-7','2-71','2-72','2-73','2-74','2-75','2-76','2-77','2-79','2-8','2-80','2-9','2-90','3','3-1','3-11','3-12','3-19','3-2','3-21','3-22','3-23','3-29','3-3','3-31','3-32','3-33','3-39','3-9','3-90','4','4-1','4-11','4-12','4-13','4-14','4-15','4-16','4-17','4-19','4-2','4-21','4-22','4-29','4-3','4-31','4-32','4-33','4-34','4-35','4-39','4-4','4-41','4-42','4-43','4-49','4-5','4-51','4-52','4-53','4-54','4-59','4-6','4-60','4-7','4-8','4-71','4-72','4-73','4-74','4-75','4-76','4-77','4-78','4-79','4-81','4-82','4-83','4-84','4-85','4-89','4-9','4-90','5','5-1','5-11','5-12','5-13','5-14','5-15','5-16','5-19','5-2','5-21','5-22','5-23','5-24','5-29','5-3','5-31','5-32','5-33','5-34','5-35','5-36','5-39','5-4','5-41','5-42','5-43','5-49','5-5','5-51','5-52','5-53','5-54','5-59','5-9','5-91','5-92','6','7','8','9','6-1','6-11','6-12','6-13','6-14','6-15','6-16','6-17','6-19','6-2','6-3','6-21','6-22','6-23','6-24','6-25','6-26','6-27','6-28','6-29','6-31','6-32','6-39','6-4','6-5','6-41','6-42','6-43','6-44','6-45','6-46','6-47','6-48','6-49','6-51','6-52','6-53','6-54','6-55','6-56','6-57','6-59','6-6','6-61','6-62','6-63','6-64','6-65','6-66','6-67','6-69','6-7','6-8','6-9','6-71','6-72','6-73','6-74','6-75','6-76','6-77','6-78','6-79','6-81','6-82','6-83','6-84','6-85','6-86','6-87','6-88','6-89','6-91','6-92','6-93','6-94','6-95','6-96','6-99','7-1','7-11','7-12','7-13','7-19','7-2','7-21','7-22','7-23','7-24','7-25','7-26','7-29','7-3','7-31','7-32','7-33','7-34','7-35','7-39','7-4','7-41','7-42','7-49','7-5','7-51','7-52','7-53','7-54','7-55','7-59','7-6','7-61','7-62','7-63','7-64','7-69','7-7','7-71','7-72','7-73','7-74','7-75','7-76','7-77','7-78','7-79','7-8','7-81','7-82','7-83','7-89','7-9','7-91','7-92','7-93','7-94','7-99','8-1','8-11','8-12','8-13','8-14','8-15','8-16','8-19','8-2','8-21','8-22','8-23','8-24','8-25','8-26','8-27','8-29','8-3','8-31','8-32','8-33','8-34','8-35','8-39','8-4','8-41','8-42','8-43','8-44','8-45','8-49','8-5','8-51','8-52','8-53','8-59','8-6','8-61','8-62','8-63','8-64','8-65','8-66','8-67','8-68','8-69','8-7','8-71','8-72','8-73','8-79','8-8','8-9','8-81','8-82','8-83','8-84','8-85','8-86','8-87','8-88','8-89','8-91','8-99','9-1','9-11','9-12','9-13','9-14','9-15','9-19','9-2','9-21','9-22','9-23','9-29','9-3','9-31','9-32','9-33','9-34','9-39','9-9','9-91','9-92','9-93','X','X-0','X-00']
        x2=['不便分类的其他从业人员','不便分类的其他从业人员','不便分类的其他从业人员','国家机关、党群组织、企业、事业单位负责人','中国共产党中央委员会和地方各级组织负责人','中国共产党中央委员会和地方各级组织负责人','国家机关及其工作机构负责人','国家权力机关及其工作机构负责人','人民政协及其工作机构负责人','人民法院负责人','人民检察院负责人','国家行政机关及其工作机构负责人','其他国家机关及其工作机构负责人','民主党派和社会团体及其工作机构负责人','民主党派负责人','工会、共青团、妇联、其他人民团体及其工作机构负责人','群众自治组织负责人','其他社会团体及其工作机构负责人','事业单位负责人','教育教学单位负责人','卫生单位负责人','科研单位负责人','其他事业单位负责人','企业负责人','企业负责人','专业技术人员','专业技术人员','科学研究人员','科学研究人员','哲学研究人员','经济学研究人员','法学研究人员','社会学研究人员','教育科学研究人员','文学、艺术研究人员','图书馆学、情报学研究人员','历史学研究人员','管理科学研究人员','数学研究人员','物理学研究人员','化学研究人员','天文学研究人员','地球科学研究人员','生物科学研究人员','农业科学研究人员','医学研究人员','其他科学研究人员','工程技术人员','工程技术人员','工程技术人员','工程技术人员','地质勘探工程技术人员','测绘工程技术人员','矿山工程技术人员','石油工程技术人员','冶金工程技术人员','化工工程技术人员','机械工程技术人员','兵器工程技术人员','航空工程技术人员','航天工程技术人员','电子工程技术人员','通信工程技术人员','计算机与应用工程技术人员','电气工程技术人员','电力工程技术人员','邮政工程技术人员','广播、电影、电视工程技术人员','交通工程技术人员','民用航空工程技术人员','铁路工程技术人员','建筑工程技术人员','建材工程技术人员','林业工程技术人员','水利工程技术人员','海洋工程技术人员','水产工程技术人员','纺织工程技术人员','食品工程技术人员','气象工程技术人员','地震工程技术人员','环境保护工程技术人员','安全工程技术人员','标准化、计量、质量工程技术人员','管理（工业）工程技术人员','其他工程技术人员','农业技术人员','土壤肥料技术人员','植物保护技术人员','园艺技术人员','作物遗传育种栽培技术人员','兽医、兽药技术人员','畜牧与草业技术人员','其他农业技术人员','飞机和船舶技术人员','飞行人员和领航人员','船舶指挥和引航人员','其他飞机和船舶技术人员','卫生专业技术人员','西医医师','中医医师','中西医结合医师','民族医生','公共卫生医师','药剂人员','医疗技术人员','护理人员','其他卫生专业技术人员','经济业务人员','经济计划人员','统计人员','会计人员','审计人员','国际商务人员','其他经济业务人员','金融业务人员','银行业务人员','保险业务人员','证券业务人员','其他金融业务人员','法律专业人员','法官','检察官','律师','公证员','司法鉴定人员','书记员','其他法律专业人员','教学人员','高等教育教师','中等职业教育教师','中学教师','小学教师','幼儿教师','特殊教育教师','其他教学人员','文学艺术工作人员','文艺创作和评论人员','编导和音乐指挥人员','演员','乐器演奏员','电影、电视制作及舞台专业人员','美术专业人员','工艺美术专业人员','其他文学艺术工作人员','体育工作人员','体育工作人员','新闻出版、文化工作人员','记者','编辑','校对员','播音员及节目主持人','翻译','图书资料与档案业务人员','考古及文物保护工作人员','其他新闻出版、文化工作人员','宗教职业者','宗教职业者','其他专业技术人员','其他专业技术人员','办事人员和有关人员','行政办公人员','行政业务人员','行政事务人员','其他行政办公人员','安全保卫和消防人员','人民警察','治安保卫人员','消防人员','其他安全保卫和消防人员','邮政和电信业务人员','邮政业务人员','电信业务人员','电信通信传输业务人员','其他邮政和电信业务人员','其他办事人员和有关人员','其他办事人员和有关人员','商业、服务业人员','购销人员','营业人员','推销、展销人员','采购人员','拍卖、典当及租赁业务人员','废旧物资回收利用人员','粮油管理人员','商品监督和市场管理人员','其他购销人员','仓储人员','保管人员','储运人员','其他仓储人员','餐饮服务人员','中餐烹饪人员','西餐烹饪人员','调酒和茶艺人员','营养配餐人员','餐厅服务人员','其他餐饮服务人员','饭店、旅游及健身娱乐场所服务人员','饭店服务人员','旅游及公共游览场所服务人员','健身和娱乐场所服务人员','其他饭店、旅游及健身娱乐场所服务人员','运输服务人员','公路、道路运输服务人员','铁路客货运输服务人员','航空运输服务人员','水上运输服务人员','其他运输服务人员','医疗卫生辅助服务人员','医疗卫生辅助服务人员','社会服务和居民生活服务人员','社会服务和居民生活服务人员','社会中介服务人员','物业管理人员','供水、供热及生活燃料供应服务人员','美容美发人员','摄影服务人员','验光配镜人员','洗染织补人员','浴池服务人员','印章刻字人员','日用机电产品维修人员','办公设备维修人员','保育、家庭服务人员','环境卫生人员','殡葬服务人员','其他社会服务和居民生活服务人员','其他商业、服务业人员','其他商业、服务业人员','农、林、牧、渔、水利业生产人员','种植业生产人员','大田作物生产人员','农业实验人员','园艺作物生产人员','热带作物生产人员','中药材生产人员','农副林特产品加工人员','其他种植业生产人员','林业生产及野生动植物保护人员','营造林人员','森林资源管护人员','野生动植物保护及自然保护区人员','木材采运人员','其他林业生产及野生动植物保护人员','畜牧业生产人员','家畜饲养人员','家禽饲养人员','蜜蜂饲养人员','实验动物饲养人员','动物疫病防治人员','草业生产人员','其他畜牧业生产人员','渔业生产人员','水产养殖人员','水产捕捞及有关人员','水产品加工人员','其他渔业生产人员','水利设施管理养护人员','河道、水库管养人员','农田灌排工程建设管理维护人员','水土保持作业人员','水文勘测作业人员','其他水利设施管理养护人员','其他农、林、牧、渔、水利业生产人员','农林专用机械操作人员','农村能源开发利用人员','生产、运输设备操作人员及有关人员','生产、运输设备操作人员及有关人员','生产、运输设备操作人员及有关人员','生产、运输设备操作人员及有关人员','勘测及矿物开采人员','地质勘查人员','测绘人员','矿物开采人员','矿物处理人员','钻井人员','石油、天然气开采人员','盐业生产人员','其他勘测及矿物开采人员','金属冶炼、轧制人员','金属冶炼、轧制人员','炼铁人员','炼钢人员','铁合金冶炼人员','重有色金属冶炼人员','轻有色金属冶炼人员','稀贵金属冶炼人员','半导体材料制备人员','金属轧制人员','铸铁管人员','炭素制品生产人员','硬质合金生产人员','其他金属冶炼、轧制人员','化工产品生产人员','化工产品生产人员','化工产品生产通用工艺人员','石油炼制生产人员','煤化工生产人员','化学肥料生产人员','无机化工产品生产人员','基本有机化工产品生产人员','合成树脂生产人员','合成橡胶生产人员','化学纤维生产人员','合成革生产人员','精细化工产品生产人员','信息记录材料生产人员','火药、炸药制造人员','林产化工产品生产人员','复合材料加工人员','日用化学品生产人员','其他化工产品生产人员','机械制造加工人员','机械冷加工人员','机械热加工人员','特种加工设备操作人员','冷作钣金加工人员','工件表面处理加工人员','磨料磨具制造加工人员','航天器件加工成型人员','其他机械制造加工人员','机电产品装配人员','机电产品装配人员','机电产品装配人员','基础件、部件装配人员','机械设备装备人员','动力设备装配人员','电气元件及设备装配人员','电子专用设备装配调试人员','仪器仪表装配人员','运输车辆装配人员','膜法水处理设备制造人员','医疗器械装配及假肢与矫形器制作人员','日用机械电器制造装配人员','五金制品制作、装配人员','装甲车辆装试人员','枪炮制造人员','弹制造人员','引信加工制造人员','火工品制造人员','防化器材制造人员','船舶制造人员','航空产品装配与调试人员','航空产品试验人员','导弹卫星装配测试人员','火箭发动机装配试验人员','航天器结构强度、温度、环境试验人员','靶场试验人员','其他机电产品装配人员','机械设备修理人员','机械设备维修人员','仪器仪表修理人员','民用航空器维修人员','其他机械设备修理人员','电力设备安装、运行、检修及供电人员','电力设备安装人员','发电运行值班人员','输电、配电、变电设备值班人员','电力设备检修人员','供用电人员','生活、生产电力设备安装、操作、修理人员','其他电力设备安装、运行、检修及供电人员','电子元器件与设备制造、装配、调试及维修人员','电子器件制造人员','电子元件制造人员','电池制造人员','电子设备装配、调试人员','电子产品维修人员','其他电子元器件与设备制造、装配、调试及维修人员','橡胶和塑料制品生产人员','橡胶制品生产人员','塑料制品加工人员','其他橡胶和塑料制品生产人员','纺织、针织、印染人员','纤维预处理人员','纺纱人员','织造人员','针织人员','印染人员','其他纺织、针织、印染人员','裁剪、缝纫和皮革、毛皮制品加工制作人员','裁剪、缝纫人员','鞋帽制作人员','皮革、毛皮加工人员','缝纫制品再加工人员','其他裁剪、缝纫和皮革、毛皮制品加工制作人员','粮油、食品、饮料生产加工及饲料生产加工人员','粮油生产加工人员','制糖和糖制品加工人员','乳品、冷食品及罐头、饮料制作人员','酿酒、食品添加剂及调味品制作人员','粮油食品制作人员','屠宰加工人员','肉、蛋食品加工人员','饲料生产加工人员','其他粮油、食品、饮料生产加工及饲料生产加工人员','烟草及其制品加工人员','原烟复烤人员','卷烟生产人员','烟用醋酸纤维丝束滤棒制作人员','其他烟草及其制品加工人员','药品生产人员','合成药物制造人员','生物技术制药（品）人员','药物制剂人员','中药制药人员','其他药品生产人员','木材加工、人造板生产、木制品制作及制浆、造纸和纸制品生产加工人员','木材加工人员','人造板生产人员','木材制品制作人员','制浆人员','造纸人员','纸制品制作人员','其他木材加工、人造板生产、木制品制作及制浆、造纸和纸制品生产加工人员','建筑材料生产加工人员','水泥及水泥制品生产加工人员','墙体屋面材料生产人员','建筑防水密封材料生产人员','建筑保温及吸音材料生产人员','装饰石材生产人员','非金属矿及其制品生产加工人员','耐火材料生产人员','其他建筑材料生产加工人员','玻璃、陶瓷、搪瓷及其制品生产加工人员','玻璃熔制人员','玻璃纤维及其制品生产人员','石英玻璃制品加工人员','陶瓷制品生产人员','搪瓷制品生产人员','其他玻璃、陶瓷、搪瓷及其制品生产加工人员','广播影视制品制作、播放及文物保护作业人员','影视制品制作人员','音像制品制作、复制人员','广播影视舞台设备安装调试及运行操作人员','电影放映人员','文物保护作业人员','其他广播影视制品制作、播放及文物保护作业人员','印刷人员','印前处理人员','印刷操作人员','印后制作人员','其他印刷人员','工艺、美术品制作人员','珠宝首饰加工制作人员','地毯制作人员','玩具制作人员','漆器工艺品制作人员','抽纱、刺绣工艺品制作人员','金属工艺品制作人员','雕刻工艺品制作人员','美术品制作人员','其他工艺、美术品制作人员','文化教育、体育用品制作人员','文教用品制作人员','体育用品制作人员','乐器制作人员','其他文化教育、体育用品制作人员','工程施工人员','工程施工人员','土石方施工人员','砌筑人员','混凝土配制及制品加工人员','钢筋加工人员','施工架子搭设人员','工程防水人员','装饰、装修人员','古建筑修建人员','筑路、养护、维修人员','工程设备安装人员','其他工程施工人员','运输设备操作人员及有关人员','公（道）路运输机械设备操作及有关人员','铁路、地铁运输机械设备操作及有关人员','民用航空设备操作及有关人员','水上运输设备操作及有关人员','起重装卸机械操作及有关人员','其他运输设备操作人员及有关人员','环境监测与废物处理人员','环境监测人员','海洋环境调查与监测人员','废物处理人员','其他环境监测与废物处理人员','检验、计量人员','检验人员','航空产品检验人员','航天器检验、测试人员','计量人员','其他检验、计量人员','其他生产、运输设备操作人员及有关人员','包装人员','机泵操作人员','简单体力劳动人员','军人','军人','军人']
        for i in range(len(x1)):
            if(x==x1[i]):
                return x2[i]
        return None

In [12]:
# 国家名称填补
def Fill_NationalName(x,y):
    if(y is not None):
        return y
    else:
        if(x is None):
            return None
        x=str(x)
        if('.' in x):
            x=x.split('.')[0]
        x1=['156','4','8','12','16','20','24','660','10','28','32','51','533','36','40','31','44','48','50','52','112','56','84','204','60','64','68','70','72','74','76','86','96','100','854','108','116','120','124','132','136','140','148','152','344','446','158','162','166','170','174','178','180','184','188','384','191','192','196','203','208','262','212','214','626','218','818','222','226','232','233','231','238','234','242','246','250','254','258','260','266','270','268','276','288','292','300','304','308','312','316','320','324','624','328','332','334','340','348','352','356','360','364','368','372','376','380','388','392','400','398','404','296','408','410','414','417','418','428','422','426','430','434','438','440','442','807','450','454','458','462','466','470','584','474','478','480','175','484','583','498','492','496','500','504','508','104','516','520','524','528','530','540','554','558','562','566','570','574','580','578','512','586','585','275','591','598','600','604','608','612','616','620','630','634','638','642','643','646','654','659','662','666','670','882','674','678','682','686','690','694','702','703','705','90','706','710','239','724','144','736','740','744','748','752','756','760','762','834','764','768','772','776','780','788','792','795','796','798','800','804','784','826','840','581','858','860','548','336','862','704','92','850','876','732','887','891','894','716']
        x2=['中国','阿富汗','阿尔巴尼亚','阿尔及利亚','美属萨摩亚','安道尔','安哥拉','安圭拉','南极洲','安提瓜和巴布达','阿根廷','亚美尼亚','阿鲁巴','澳大利亚','奥地利','阿塞拜疆','巴哈马','巴林','孟加拉国','巴巴多斯','白俄罗斯','比利时','伯利兹','贝宁','百慕大','不丹','玻利维亚','波黑','博茨瓦纳','布维岛','巴西','英属印度洋领地','文莱','保加利亚','布基纳法索','布隆迪','柬埔寨','喀麦隆','加拿大','佛得角','开曼群岛','中非','乍得','智利','香港','澳门','台湾','圣诞岛','科科斯（基林）群岛','哥伦比亚','科摩罗','刚果（布）','刚果（金）','库克群岛','哥斯达黎加','科特迪瓦','克罗地亚','古巴','塞浦路斯','捷克','丹麦','吉布提','多米尼克','多米尼加','东帝汶','厄瓜多尔','埃及','萨尔瓦多','赤道几内亚','厄立特里亚','爱沙尼亚','埃塞俄比亚','福克兰群岛（马尔维纳斯）','法罗群岛','斐济','芬兰','法国','法属圭亚那','法属波利尼西亚','法属南部领地','加蓬','冈比亚','格鲁吉亚','德国','加纳','直布罗陀','希腊','格陵兰','格林纳达','瓜德罗普','关岛','危地马拉','几内亚','几内亚比绍','圭亚那','海地','赫德岛和麦克唐纳岛','洪都拉斯','匈牙利','冰岛','印度','印度尼西亚','伊朗','伊拉克','爱尔兰','以色列','意大利','牙买加','日本','约旦','哈萨克斯坦','肯尼亚','基里巴斯','朝鲜','韩国','科威特','吉尔吉斯斯坦','老挝','拉脱维亚','黎巴嫩','莱索托','利比里亚','利比亚','列支敦士登','立陶宛','卢森堡','前南巴其顿','马达加斯加','马拉维','马来西亚','马尔代夫','马里','马耳他','马绍尔群岛','马提尼克','毛里塔尼亚','毛里求斯','马约特','墨西哥','密克罗尼西亚联邦','摩尔多瓦','摩纳哥','蒙古','蒙特塞拉特','摩洛哥','莫桑比克','缅甸','纳米比亚','瑙鲁','尼泊尔','荷兰','荷属安的列斯','新喀里多尼亚','新西兰','尼加拉瓜','尼日尔','尼日利亚','纽埃','诺福克岛','北马里亚纳','挪威','阿曼','巴基斯坦','帕劳','巴勒斯坦','巴拿马','巴布亚新几内亚','巴拉圭','秘鲁','菲律宾','皮特凯恩','波兰','葡萄牙','波多黎各','卡塔尔','留尼汪','罗马尼亚','俄罗斯联邦','卢旺达','圣赫勒拿','圣基茨和尼维斯','圣卢西亚','圣皮埃尔和密克隆','圣文森特和格林纳丁斯','萨摩亚','圣马力诺','圣多美和普林西比','沙特阿拉伯','塞内加尔','塞舌尔','塞拉利昂','新加坡','斯洛伐克','斯洛文尼亚','所罗门群岛','索马里','南非','南乔治亚岛和南桑德韦奇岛','西班牙','斯里兰卡','苏丹','苏里南','斯瓦尔巴岛和扬马延岛','斯威士兰','瑞典','瑞士','叙利亚','塔吉克斯坦','坦桑尼亚','泰国','多哥','托克劳','汤加','特立尼克和多巴哥','突尼斯','土耳其','土库曼斯坦','特克斯和凯科斯群岛','图瓦卢','乌干达','乌克兰','阿联酋','英国','美国','美国本土外小岛屿','乌拉圭','乌兹别克斯坦','瓦努阿图','梵蒂冈','委内瑞拉','越南','英属维尔京群岛','美属维尔京群岛','瓦利斯和富图纳','西撒哈拉','也门','南斯拉夫','赞比亚','津巴布韦']
        for i in range(len(x1)):
            if(x==x1[i]):
                return x2[i]
        return None

In [13]:
# 身份证类别名称填补
def Fill_IDType(x,y):
    if(y is not None):
        return y
    else:
        if(x is None):
            return None
        x=str(x)
        if('.' in x):
            x=x.split('.')[0]
        x1=['1','2','3','4','5','6','7','19','99']
        x2=['居民身份证','居民户口簿','护照','军官证','驾驶证','港澳居民来往内地通行证','台湾居民来往内地通行证','母亲身份证','其他法定有效证件']
        for i in range(len(x1)):
            if(x==x1[i]):
                return x2[i]
        return None

In [14]:
# 学历名称填补
def Fill_EducationLevelName(x,y):
    if(y is not None):
        return y
    else:
        if(x is None):
            return None
        x=str(x)
        if('.' in x):
            x=x.split('.')[0]
        x1=['11','12','13','14','15','16','17','18','19','20','21','22','23','28','30','31','32','33','40','41','42','43','44','45','46','47','48','49','60','61','62','63','7','70','71','73','8','80','81','83','9','90']
        x2=['博士研究生毕业','博士研究生结业','博士研究生肄业','硕士研究生毕业','硕士研究生结业','硕士研究生肄业','研究生班毕业','研究生班结业','研究生班肄业','大学本科','大学本科毕业','大学本科结业','大学本科肄业','大学普通班毕业','大学专科教育','大学专科毕业','大学专科结业','大学专科肄业','中等职业教育','中等专科毕业','中等专科结业','中等专科肄业','职业高中毕业','职业高中结业','职业高中肄业','技工学校毕业','技工学校结业','技工学校肄业','普通高级中学教育','普通高中毕业','普通高中结业','普通高中肄业','初级中学教育','初级中学教育','初中毕业','初中肄业','小学','小学教育','小学毕业','小学肄业','其他','其他']
        for i in range(len(x1)):
            if(x==x1[i]):
                return x2[i]
        return None

In [1]:
# 构建函数
def Build(filelist,output_path,name):
    # 初始化环境
    spark = SparkSession.builder.appName('A1_Create').config('spark.dynamicAllocation.enabled','true').config('spark.debug.maxToStringFields', '100').config('spark.sql.execution.arrow.enable','true').config('spark.executor.memory','48g').config('spark.driver.memory', '48g').config('spark.core.connection.ack.wait.timeout','36000s').config('spark.executor.heartbeatInterval','36000s').config('spark.network.timeout', '50000s').config('spark.rpc.lookupTimeout', '5000s').config('spark.shuffle.io.connectionTimeout', '50000s').getOrCreate()
    files=[]
    start=time()
    # 读入数据
    print('开始处理分块文件')
    for file in filelist:
        files.append(spark.read.option("inferSchema","true").option("delimiter", "\t").option("header","false").csv(file))
    print('读入数据耗时：%.2f秒'%(time()-start))
    # 合并数据
    start=time()
    reduce(lambda x,y:x.unionByName(y), files).createOrReplaceTempView('A1_Raw')
    # 注册函数
    spark.udf.register('Birth',Birth,DateType())
    spark.udf.register('Gender',Gender,StringType())
    spark.udf.register('GenderName',GenderName,StringType())
    spark.udf.register('Address',Address,StringType())
    spark.udf.register('Fill_Address',Fill_Address,StringType())
    spark.udf.register('Cut_Space',Cut_Space,StringType())
    spark.udf.register('Fill_MaritalStatusName',Fill_MaritalStatusName,StringType())
    spark.udf.register('Fill_RelationshipName',Fill_RelationshipName,StringType())
    spark.udf.register('Fill_OccupationalCategoryName',Fill_OccupationalCategoryName,StringType())
    spark.udf.register('Fill_NationalName',Fill_NationalName,StringType())
    spark.udf.register('Fill_IDType',Fill_IDType,StringType())
    spark.udf.register('Fill_EducationLevelName',Fill_EducationLevelName,StringType())
    # 清洗数据
    spark.sql('''
    select _c0 as MEDICAL_INSTITUTION_CODE
    ,Cut_Space(_c1) as MEDICAL_INSTITUTION_NAME
    ,_c2 as FILE_NUMBER
    ,_c3 as HEALTH_RECORD_NUMBER
    ,_c4 as HEALTH_CARD_NUMBER
    ,_c5 as CARD_NUMBER
    ,_c6 as CARD_TYPE
    ,_c7 as CARD_TYPE_NAME
    ,Cut_Space(_c8) as ID_NUMBER
    ,_c9 as ID_TYPE
    ,Fill_IDType(_c9,_c10) as ID_TYPE_NAME
    ,Cut_Space(_c11) as NAME
    ,Birth(to_date(_c12),to_date(_c58)) as BIRTH_DATE
    ,Gender(Cut_Space(_c13),Cut_Space(_c14)) as GENDER
    ,GenderNAME(Cut_Space(_c14),Cut_Space(_c13)) as GENDER_NAME
    ,Cut_Space(_c15) as MARITAL_STATUS
    ,Fill_MaritalStatusName(Cut_Space(_c15),Cut_Space(_c16)) as MARITAL_STATUS_NAME
    ,Cut_Space(_c17) as NATION
    ,Cut_Space(_c18) as NATION_NAME
    ,Cut_Space(_c19) as NATIONALITY
    ,Fill_NationalName(Cut_Space(_c19),Cut_Space(_c20)) as NATIONALITY_NAME
    ,Cut_Space(_c21) as OCCUPATIONAL_CATEGORY
    ,Fill_OccupationalCategoryName(Cut_Space(_c21),Cut_Space(_c22)) as OCCUPATIONAL_CATEGORY_NAME
    ,Cut_Space(_c23) as EDUCATION_LEVEL
    ,Fill_EducationLevelName(Cut_Space(_c23),Cut_Space(_c24)) as EDUCATION_LEVEL_NAME
    ,Cut_Space(_c25) as TELEPHONE_NUMBER
    ,Cut_Space(_c26) as MOBILE_PHONE_NUMBER
    ,Cut_Space(_c27) as EMAIL_ADDRESS
    ,Cut_Space(_c28) as NAME_OF_EMPLOYER
    ,Cut_Space(_c29) as PHONE_OF_EMPLOYER
    ,Address(Cut_Space(_c30)) as ADDRESS_OF_EMPLOYER
    ,Cut_Space(_c31) as POSTCODE_OF_EMPLOYER
    ,Cut_Space(_c32) as CONTACT_PERSONS_NAME
    ,Cut_Space(_c33) as CONTACT_PERSONS_PHONE
    ,Cut_Space(_c34) as CONTACT_ID_NUMBER
    ,Cut_Space(_c35) as RELATIONSHIP_WITH_CONTACT
    ,Fill_RelationshipName(Cut_Space(_c35),Cut_Space(_c36)) as RELATIONSHIP_NAME
    ,Address(Cut_Space(_c37)) as CONTACT_PERSONS_ADDRESS
    ,Cut_Space(_c38) as CONTACT_PERSONS_POSTCODE
    ,Cut_Space(_c39) as RESIDENCE_SIGN
    ,Fill_Address(Address(Cut_Space(_c40)),Address(Cut_Space(concat_ws('',_c42,_c43,_c44,_c45,_c46,_c47)))) as RESIDENCE_ADDRESS
    ,Cut_Space(_c41) as DIVISIONS_CODE_OF_RESIDENCE
    ,Cut_Space(_c42) as PROVINCE_OF_RESIDENCE
    ,Cut_Space(_c43) as CITY_OF_RESIDENCE
    ,Cut_Space(_c44) as COUNTY_OF_RESIDENCE
    ,Cut_Space(_c45) as TOWN_OF_RESIDENCE
    ,Cut_Space(_c46) as VILLAGE_OF_RESIDENCE
    ,Cut_Space(_c47) as HOUSE_NUMBER_OF_RESIDENCE
    ,Cut_Space(_c48) as POSTCODE_OF_RESIDENCE
    ,Fill_Address(Address(Cut_Space(_c49)),Address(Cut_Space(concat_ws('',_c51,_c52,_c53,_c54,_c55,_c56)))) as REGISTERED_ADDRESS
    ,Cut_Space(_c50) as DIVISIONS_CODE_REG_RESIDENCE
    ,Cut_Space(_c51) as PROVINCE_OF_REG_RESIDENCE
    ,Cut_Space(_c52) as CITY_OF_REG_RESIDENCE
    ,Cut_Space(_c53) as COUNTY_OF_REG_RESIDENCE
    ,Cut_Space(_c54) as TOWN_OF_REG_RESIDENCE
    ,Cut_Space(_c55) as VILLAGE_OF_REG_RESIDENCE
    ,Cut_Space(_c56) as HOUSE_NUMBER_OF_REG_RESIDENCE
    ,Cut_Space(_c57) as POSTCODE_OF_REG_RESIDENCE
    ,_c58 as ETL_DATE
    ,Address(Cut_Space(_c59)) as AREA
    ,to_date(_c60) as id_birth_date
    ,_c61 as id_gender
    from A1_Raw
    ''').coalesce(1).write.csv(output_path+name,mode='overwrite',header=True)
    # 释放资源
    spark.catalog.dropTempView("A1_Raw")
    shutil.move(GetPath(output_path+name+'/','csv'),output_path+name+'.csv')
    shutil.rmtree(output_path+name)
    print('清洗数据耗时：%.2f秒'%(time()-start))
    print(f'已输出至：{output_path}{name}.csv')


In [2]:
def Create_A1(input_path,output_path):
    t_start=time()
    filelist=GetPath(input_path,'tb_patient_information')
    Build(filelist[:54],output_path,'tb_patient_information')
    Build(filelist[54:],output_path,'tb_patient_information_201807')
    print('总耗时：%.2f秒'%(time()-t_start))

In [ ]:
if __name__=="__main__":
    Create_A1(input_path,output_path)